In [6]:
import pandas as pd
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

class SyntheticDataGenerator:
    """
    Générateur de données synthétiques pour machine learning
    Préserve les distributions statistiques des données originales
    EN COLLABORATION AVEC NOTRE CHER AMI CHATGPT
    """
    
    def __init__(self, filepath):
        """
        Initialise le générateur avec les données originales
        
        Args:
            filepath: Chemin vers le fichier CSV (délimiteur point-virgule)
        """
        self.df_original = pd.read_csv(filepath, delimiter=';', encoding='utf-8')
        self.stats = {}
        self._analyze_data()
        
    def _analyze_data(self):
        """Analyse les distributions statistiques des données"""
        print(f" Analyse de {len(self.df_original)} lignes...")
        print(f" Colonnes: {list(self.df_original.columns)}\n")
        
        for col in self.df_original.columns:
            if self.df_original[col].dtype == 'object' or pd.api.types.is_categorical_dtype(self.df_original[col]):
                # Variable catégorielle
                value_counts = self.df_original[col].value_counts(dropna=True)
                distribution = value_counts / value_counts.sum()
                probabilities = list(distribution.values)
                # Normaliser pour s'assurer que la somme = 1
                probabilities = np.array(probabilities)
                probabilities = probabilities / probabilities.sum()
                self.stats[col] = {
                    'type': 'categorical',
                    'values': list(value_counts.index),
                    'probabilities': list(probabilities)
                }
                print(f"✓ {col} (catégorielle): {len(value_counts)} valeurs uniques")
            else:
                # Variable numérique
                self.stats[col] = {
                    'type': 'numeric',
                    'mean': self.df_original[col].mean(),
                    'std': self.df_original[col].std(),
                    'min': self.df_original[col].min(),
                    'max': self.df_original[col].max()
                }
                print(f" {col} (numérique): μ={self.stats[col]['mean']:.2f}, σ={self.stats[col]['std']:.2f}")
    
    def generate(self, n_rows=1000):
        """
        Génère des données synthétiques
        
        Args:
            n_rows: Nombre de lignes à générer
            
        Returns:
            DataFrame avec les données synthétiques
        """
        print(f"\n Génération de {n_rows} lignes synthétiques...")
        
        synthetic_data = {}
        
        for col, stat in self.stats.items():
            if stat['type'] == 'categorical':
                # Échantillonnage selon la distribution d'origine
                synthetic_data[col] = np.random.choice(
                    stat['values'],
                    size=n_rows,
                    p=stat['probabilities']
                )
            else:
                # Distribution normale avec les paramètres d'origine
                values = np.random.normal(
                    loc=stat['mean'],
                    scale=stat['std'],
                    size=n_rows
                )
                # Limiter aux min/max d'origine
                values = np.clip(values, stat['min'], stat['max'])
                # Arrondir à 2 décimales
                synthetic_data[col] = np.round(values, 2)
        
        df_synthetic = pd.DataFrame(synthetic_data)
        print(f"Génération terminée!")
        
        return df_synthetic
    
    def save(self, df_synthetic, output_path='synthetic_data.csv'):
        """
        Sauvegarde les données synthétiques en CSV
        
        Args:
            df_synthetic: DataFrame à sauvegarder
            output_path: Chemin du fichier de sortie
        """
        df_synthetic.to_csv(output_path, sep=';', index=False, encoding='utf-8')
        print(f"Fichier sauvegardé: {output_path}")
        print(f"Dimensions: {df_synthetic.shape[0]} lignes × {df_synthetic.shape[1]} colonnes")
    
    def compare_distributions(self, df_synthetic):
        """
        Compare les distributions originales et synthétiques
        
        Args:
            df_synthetic: DataFrame synthétique à comparer
        """
        print("\n COMPARAISON DES DISTRIBUTIONS")
        print("=" * 60)
        
        for col in self.df_original.columns:
            if self.stats[col]['type'] == 'numeric':
                orig_mean = self.df_original[col].mean()
                synth_mean = df_synthetic[col].mean()
                orig_std = self.df_original[col].std()
                synth_std = df_synthetic[col].std()
                
                print(f"\n{col}:")
                print(f"  Moyenne  → Original: {orig_mean:.2f} | Synthétique: {synth_mean:.2f}")
                print(f"  Écart-type → Original: {orig_std:.2f} | Synthétique: {synth_std:.2f}")
            else:
                print(f"\n{col} (catégorielle):")
                orig_dist = self.df_original[col].value_counts(normalize=True)
                synth_dist = df_synthetic[col].value_counts(normalize=True)
                
                for val in orig_dist.index[:3]:  # Top 3 valeurs
                    orig_pct = orig_dist.get(val, 0) * 100
                    synth_pct = synth_dist.get(val, 0) * 100
                    print(f"  {val} → Original: {orig_pct:.1f}% | Synthétique: {synth_pct:.1f}%")


# -_-_-_-_-_-_-_-_-_-_-_
# EXEMPLE D'UTILISATION
# -_-_-_-_-_-_-_-_-_-_-_

if __name__ == "__main__":
    print("=" * 22)
    print(" GÉNÉRATEUR DE DONNÉES SYNTHÉTIQUES POUR ML")
    print("=" * 22)
    print()
    
    # 1. Charger et analyser les données originales
    generator = SyntheticDataGenerator('dataset_final.csv')
    
    # 2. Générer des données synthétiques
    n_rows = 1000  # Modifiez ce nombre selon vos besoins
    df_synthetic = generator.generate(n_rows=n_rows)
    
    # 3. Aperçu des données générées
    print("\n APERÇU DES DONNÉES GÉNÉRÉES")
    print("=" * 60)
    print(df_synthetic.head(10))
    
    # 4. Sauvegarder
    generator.save(df_synthetic, output_path=f'synthetic_2data_{n_rows}.csv')
    
    # 5. Comparer les distributions
    generator.compare_distributions(df_synthetic)
    
    print("\n" + "=" * 22)
    print("TERMINÉ! Nos données synthétiques sont prêtes pour le ML!")
    print("=" * 22)

🚀 GÉNÉRATEUR DE DONNÉES SYNTHÉTIQUES POUR ML

📊 Analyse de 276 lignes...
📋 Colonnes: ['sexe', 'niveau', 'sport', 'E. Social', 'repos', 'assistance', 'pratique', 'qualité', 'organisation', 'assiduite', 'facteurs +', 'revision', 'note']

✓ sexe (numérique): μ=0.40, σ=0.90
✓ niveau (numérique): μ=2.29, σ=1.20
✓ sport (numérique): μ=0.31, σ=0.74
✓ E. Social (numérique): μ=0.69, σ=0.44
✓ repos (numérique): μ=5.90, σ=1.70
✓ assistance (numérique): μ=0.53, σ=0.13
✓ pratique (numérique): μ=0.14, σ=0.99
✓ qualité (numérique): μ=0.25, σ=0.51
✓ organisation (numérique): μ=0.01, σ=0.82
✓ assiduite (numérique): μ=0.56, σ=0.51
✓ facteurs + (numérique): μ=1.19, σ=0.97
✓ revision (catégorielle): 41 valeurs uniques
✓ note (catégorielle): 66 valeurs uniques

🔄 Génération de 1000 lignes synthétiques...
✅ Génération terminée!

👀 APERÇU DES DONNÉES GÉNÉRÉES
   sexe  niveau  sport  E. Social  repos  assistance  pratique  qualité  \
0  0.83    1.00   0.55       0.38   5.75        0.62      1.00    -0.17   
1